### This notebook shows how to convert from LGBM to ONNX and provides an example of the ONNXML converter

In [1]:
# Uncomment and run this cell if you need to install Hummingbird and ONNX
#!pip install hummingbird_ml[extra,onnx]

In [2]:
import torch
import numpy as np
import lightgbm as lgb

import onnxruntime as ort
from onnxmltools.convert import convert_lightgbm
from onnxconverter_common.data_types import FloatTensorType

from hummingbird.ml import convert
from hummingbird.ml import constants

# Create some random data for binary classification.
num_classes = 2
X = np.array(np.random.rand(10000, 28), dtype=np.float32)
y = np.random.randint(num_classes, size=10000)

In [3]:
# Create and train a model (LightGBM in this case).
model = lgb.LGBMClassifier()
model.fit(X, y)

LGBMClassifier()

In [4]:
# Use ONNXMLTOOLS to convert the model to ONNXML.
initial_types = [("input", FloatTensorType([X.shape[0], X.shape[1]]))] # Define the inputs for the ONNX
onnx_ml_model = convert_lightgbm(
    model, initial_types=initial_types, target_opset=9
)

In [5]:
# Use Hummingbird to convert the ONNXML model to ONNX.
onnx_model = convert(onnx_ml_model, "onnx")

In [6]:
%%timeit -r 3

# Run the ONNX model on CPU 
onnx_model.predict(X)

507 ms ± 16.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)
